In [54]:
# https://www.geeksforgeeks.org/python-sqlite-creating-a-new-database/#

from sqlalchemy import create_engine

from sqlalchemy.orm import declarative_base, Session, relationship

from sqlalchemy import Column, Integer, String, Float, DateTime, ForeignKey

import sqlite3 

from datetime import datetime

In [55]:
food_prices_db = "food_prices.sqlite"
  
try: 
  conn = sqlite3.connect(food_prices_db) 
  print(f"Database {food_prices_db} formed.") 
except: 
  print(f"Database {food_prices_db} not formed.")

Database food_prices.sqlite formed.


In [56]:
Base = declarative_base()

In [57]:
class Country(Base):
    __tablename__ = 'countries'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(56))
    code = Column(String(3))
    currency = Column(String(3))
    food_products = relationship("Food", back_populates="country")

In [58]:
class Food(Base):
    __tablename__ = 'food_prices'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(20))
    date = Column(DateTime)
    year = Column(Integer)
    month = Column(Integer)
    open = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    inflation = Column(Float)
    trust = Column(Float)
    country_id = Column(Integer, ForeignKey("countries.id"))
    country = relationship("Country", back_populates="food_products")

In [59]:
country = Country(
    name="Afghanistan", 
    code="AFG",
    currency = "AFN"
    )

In [60]:
food_price = Food(
    name = "Bread",
    date = datetime.strptime("2017-01-01", "%Y-%m-%d").date(),
    year = 2017,
    month = 1,
    open = 32.43,
    high = 32.65,
    low = 32.03,
    close = 32.4,
    inflation = 29.37,
    trust = 7.4,
    country = country
)

In [61]:
engine = create_engine("sqlite:///food_prices.sqlite")
conn = engine.connect()

In [62]:
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [63]:
session.add(country)
session.add(food_price)
session.commit()

In [64]:
country_list = session.query(Country)
for country in country_list:
    print(country.name)

Afghanistan


In [65]:
food_list = session.query(Food)
for food in food_list:
    print(food.name)

Bread


In [67]:
Base.metadata.drop_all(engine)
conn.close()